In [1]:
#import yaml
from ruamel.yaml import YAML
import pathlib, subprocess
from pathlib import Path

In [2]:
def load_with_pre_directives_comments(yaml, path):
    comments = []
    text = path.read_text()
    if '\n---\n' not in text and '\n--- ' not in text:
         return yaml.load(text), comments
    for line in text.splitlines(True):
        if line.lstrip().startswith('#'):
            comments.append(line)
        elif line.startswith('---'):
            return yaml.load(text), comments
            break

In [3]:
def lookup(sk, d, path=[]):
    # lookup the values for key(s) sk return as list the tuple (path to the value, value)
    if isinstance(d, dict):
        for k, v in d.items():
            if k == sk:
                yield (d, v)
            for res in lookup(sk, v, path + [k]):
                yield res
    elif isinstance(d, list):
        for item in d:
            for res in lookup(sk, item, path + [item]):
                yield res

In [4]:
with open('recipe_obs4mips.yml', 'r') as file:
    yaml = YAML(typ='safe')
    recipes = yaml.load(file)["recipes"]

In [14]:
for recipe in  recipes.keys():
    datasets_to_replace = recipes[recipe]["datasets"]

    yaml = YAML()
    yaml.width = 80
    yaml.indent(mapping=2, sequence=4, offset=2)
    yaml.explicit_start = True
    yaml.allow_duplicate_keys = True
    _, comments = load_with_pre_directives_comments(yaml, pathlib.Path(f"recipes/{recipe}"))
    
    with open(f"recipes/{recipe}", "r") as file:
        data = yaml.load(file)

    for key, val in datasets_to_replace.items():
        for d, value in lookup("additional_datasets", data):
            for entry in d["additional_datasets"]:
                if entry["dataset"] == key:
                    entry["dataset"] = datasets_to_replace[key]["esgf_dataset"]
                    if "version" in entry.keys():
                        entry.pop("version")
            
        for d, value in lookup("datasets", data):
            for entry in d["datasets"]:
                if entry["dataset"] == key:
                    entry["dataset"] = datasets_to_replace[key]["esgf_dataset"]
                    if "version" in entry.keys():
                        entry.pop("version")
    
    file_in = Path(recipe)    
    with open(f"{file_in.stem}.yml", 'w') as file:
        file.write(''.join(comments))   
    with open(f"{file_in.stem}.yml", 'a') as file:
        yaml.dump(data, file)

TypeError: YAML.dump() got an unexpected keyword argument 'width'